In [ ]:
import time
import logging
import sys
import utils.general as general_util
import utils.tournament_helper as tournament_helper
from dotenv import load_dotenv

In [49]:
tournament_id = "wEuVhT9c"
tournament_json, api_endpoint = tournament_helper.call_lichess_broadcasts_api(
    tournament_id
)
lichess_tour_df, lichess_rounds_df = tournament_helper.create_broadcast_base_dataframes(
    tournament_json, tournament_id
)

In [51]:
lichess_tour_df

tournament_id,name,slug,created_at,tier,image,url,leaderboard
str,str,str,i64,i64,str,str,bool
"""wEuVhT9c""","""FIDE Candidate…","""fide-candidate…",1710759962579,5,"""https://image.…","""https://liches…",true


In [52]:
lichess_rounds_df

round_id,tournament_id,name,slug,created_at,starts_at,url,finished
str,str,str,str,i64,i64,str,bool
"""AjqSsU1w""","""wEuVhT9c""","""Round 1""","""round-1""",1710760290779,1712256300000,"""https://liches…",true
"""GenKIJ8A""","""wEuVhT9c""","""Round 2""","""round-2""",1711530504000,1712342700000,"""https://liches…",true
"""xQgaUu2y""","""wEuVhT9c""","""Round 3""","""round-3""",1711530516119,1712429100000,"""https://liches…",true
"""CPS9dENa""","""wEuVhT9c""","""Round 4""","""round-4""",1711530526339,1712515500000,"""https://liches…",true
"""MDiLWQ5M""","""wEuVhT9c""","""Round 5""","""round-5""",1711530555192,1712688300000,"""https://liches…",true
…,…,…,…,…,…,…,…
"""46ohJ8Qt""","""wEuVhT9c""","""Round 11""","""round-11""",1711530640577,1713379500000,"""https://liches…",null
"""eJghIBZe""","""wEuVhT9c""","""Round 12""","""round-12""",1711530651369,1713465900000,"""https://liches…",null
"""Dc0DQMaQ""","""wEuVhT9c""","""Round 13""","""round-13""",1711530662643,1713638700000,"""https://liches…",null


In [34]:
# with open("data.json", "w") as f:
#     json.dump(tournament_json, f)